<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Case-study-description" data-toc-modified-id="Case-study-description-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Case study description</a></span></li><li><span><a href="#Read-in-the-data-set" data-toc-modified-id="Read-in-the-data-set-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Read in the data set</a></span></li><li><span><a href="#Explore-data:-show-some-tables-and-plots" data-toc-modified-id="Explore-data:-show-some-tables-and-plots-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Explore data: show some tables and plots</a></span></li><li><span><a href="#Load-another-dataset;-merge-the-two-data-tables" data-toc-modified-id="Load-another-dataset;-merge-the-two-data-tables-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Load another dataset; merge the two data tables</a></span></li><li><span><a href="#Choose-a-column-and-predict-another-column,-based-on-a-least-squares-model" data-toc-modified-id="Choose-a-column-and-predict-another-column,-based-on-a-least-squares-model-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Choose a column and predict another column, based on a least-squares model</a></span></li><li><span><a href="#Quantify-how-good-the-predictions-are" data-toc-modified-id="Quantify-how-good-the-predictions-are-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Quantify how good the predictions are</a></span></li><li><span><a href="#Try-different-prediction-models" data-toc-modified-id="Try-different-prediction-models-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Try different prediction models</a></span></li><li><span><a href="#User-interface-to-test-our-predictions" data-toc-modified-id="User-interface-to-test-our-predictions-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>User-interface to test our predictions</a></span></li><li><span><a href="#Export-the-results" data-toc-modified-id="Export-the-results-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Export the results</a></span></li><li><span><a href="#Try-some-Python-commands-yourself" data-toc-modified-id="Try-some-Python-commands-yourself-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Try some Python commands yourself</a></span></li><li><span><a href="#Next-steps-in-learning-Python" data-toc-modified-id="Next-steps-in-learning-Python-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Next steps in learning Python</a></span></li></ul></div>

# Details about this notebook

In this notebook we demonstrate some Python usage, specifically around data analysis. We consider topics that you would do on a regular basis:

* Reading in the data
* Get a bit of an understanding of the data format
* Merging it with another data set
* Building a prediction model from your data
* Exporting the results and sharing them with your colleagues.

## Case study description

Goes here

In [ ]:
import numpy as np
import pandas as pd
import plotly
pd.options.plotting.backend = "plotly"
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "notebook" # jupyterlab

## Read in the data set

In [ ]:
spectra = pd.read_excel("https://github.com/kgdunn/process-improve/raw/main/notebooks_examples/Tablets.xlsx", sheet_name="Spectra").set_index("Sample")
print(f"Data shape = {spectra.shape}")

## Explore data: show some tables and plots

In [ ]:
# Show the top of the data set
spectra.head()

In [ ]:
# Show the end of the data set
spectra.tail()

In [ ]:
# Randomly select and show 10 rows
spectra.sample(10)

In [ ]:
# Show a randomly selected row; and plot its spectrum
spectra.sample(1).iloc[0].plot(title="Plot of a randomly selected spectrum")

In [ ]:
# Improve the figure; drop the first column away
fig=spectra.sample(1).iloc[0, 1:].plot(title="Plot of a randomly selected spectrum")
fig.update_layout(xaxis_title_text="Wavelength [nm]")
fig.update_layout(yaxis_title_text="Absorbance")

## Load another dataset; merge the two data tables

In [ ]:
output = pd.read_excel("Tablets.xlsx", sheet_name="Hardness").set_index("Sample")
print(f"The 'outputs' data frame has shape = {output.shape}")

In [ ]:
output.head()

In [ ]:
# Explore the outputs
display(output['Hardness'].plot.line())
display(output['Hardness'].plot.hist(nbins=30))
display(output['Hardness'].plot.box())

In [ ]:
# Summary statistics for each column
display(output.mean())
display(output.median())
display(output.std())
display(output.min())

In [ ]:
# Get a complete summary
output.describe()

In [ ]:
spectra.index

In [ ]:
output.index

In [ ]:
# Join the two data sets
joined =spectra.merge(output, left_index=True, right_index=True)
joined.columns


In [ ]:
# Plot the output, sorted:
joined["Hardness"].sort_values()
joined["Hardness"].sort_values().plot()

In [ ]:
# See the 4 groups, based on 'Category', for the output variable
joined.groupby(["Category"])["Hardness"].mean()

In [ ]:
# Now, repeat, but for the spectra
joined.groupby(["Category"]).mean()

In [ ]:
spectra.groupby(["Category"]).mean().T.plot()

In [ ]:
# Select a column for a particular wavelength
wavelength  = '1884nm'
fig=joined.loc[:, wavelength].plot(title=f"Plot of absorbances for all tablets at wavelength {wavelength}")
fig.update_layout(xaxis_title_text="Sample number")
fig.update_layout(yaxis_title_text=f"Absorbance at {wavelength}")

## Choose a column and predict another column, based on a least-squares model

In [ ]:
# Correlation plot at a particular wavelength against Hardness
# Choose a column as x, to predict another column as y, based on a least-squares model

wavelength  = "1666nm"
two_columns = joined.loc[:, [wavelength, 'Hardness']]
# display(two_columns)
fig=two_columns.plot.scatter(x=wavelength, y='Hardness')
fig.update_layout(xaxis_title_text=f"Absorbance at {wavelength}")
fig.update_layout(yaxis_title_text="Hardness")

In [ ]:
all_correlations = joined.corr()['Hardness']

In [ ]:
all_correlations.apply(lambda x: x**2).plot()


In [ ]:
display(  all_correlations.max()          )
display(  all_correlations[0:-1].max()    )
display(  all_correlations[0:-1].argmax() )

In [ ]:
# So wavelength in position 533 is the largest: corresponds to wavelength ...
maximum_column = 533
joined.columns[maximum_column]

In [ ]:
best_wavelength = "1664nm"
joined.loc[:, best_wavelength]

In [ ]:
## Choose a column and predict another column, based on a least-squares model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

mymodel = LinearRegression()

In [ ]:
X = joined.loc[:, [best_wavelength]]
mymodel.fit(X, y=joined["Hardness"]);

In [ ]:
# The coefficients
print(f'Intercept = {mymodel.intercept_} and slope = {mymodel.coef_}')

# The mean squared error:
actual_y_values = joined["Hardness"]
predicted_y_values = mymodel.predict(X)
prediction_error = actual_y_values - predicted_y_values    
prediction_error.hist(nbins=40)

In [ ]:
joined["Predicted hardness"] = mymodel.intercept_ + mymodel.coef_ * joined[best_wavelength]

In [ ]:
# Plot the regression model, and fit
three_columns = joined.loc[:, [best_wavelength, 'Hardness', "Predicted hardness"]]
fig=three_columns.plot.scatter(x=best_wavelength, y='Hardness')
fig.update_layout(xaxis_title_text=f"Absorbance at {wavelength}")
fig.update_layout(yaxis_title_text="Hardness")
fig.add_scatter(x=three_columns[best_wavelength], y=three_columns['Predicted hardness'], name="Prediction")

## Quantify how good the predictions are

In [ ]:
print(f'Mean squared error: {mean_squared_error(actual_y_values, predicted_y_values, squared=False):.4g}')
      
# The coefficient of determination: (R^2)
print(f'Coefficient of determination = R^2 = {r2_score(actual_y_values, predicted_y_values):.3f}')

## Try different prediction models

In [ ]:
## Calculation: use the average of some columns around the best column
spectra.loc[:, "1654nm":"1674nm"]    

In [ ]:
avg_model = LinearRegression()
X = spectra.loc[:, "1654nm":"1674nm"] #.mean(axis=1).values
avg_model.fit(X, y=joined["Hardness"]);

In [ ]:
# The coefficients
print(f'Intercept = {avg_model.intercept_} and slope = {avg_model.coef_}')

# The mean squared error:
predicted_y_values_avgmodel = avg_model.predict(X)
prediction_error_avgmodel = actual_y_values - predicted_y_values_avgmodel    
prediction_error_avgmodel.hist(nbins=40)

In [ ]:
print(f'Mean squared error (single): {mean_squared_error(actual_y_values, predicted_y_values, squared=False):.4g}')
print(f'Mean squared error (multiple): {mean_squared_error(actual_y_values, predicted_y_values_avgmodel, squared=False):.4g}')

# The coefficient of determination: (R^2)
print(f'Coefficient of determination (single) = R^2 = {r2_score(actual_y_values, predicted_y_values):.3f}')
print(f'Coefficient of determination (multiple) = R^2 = {r2_score(actual_y_values, predicted_y_values_avgmodel):.3f}')

In [ ]:
#joined["Predicted hardness (multiple)"] = 
#avg_model.intercept_ + 
pd.Series(avg_model.coef_).plot()# * X

In [ ]:
from process_improve.multivariate.methods import PLS, MCUVScaler

In [ ]:
plsmodel = PLS(n_components=3)
X_mcuv = MCUVScaler().fit(X)
plsmodel.fit(X_mcuv.transform(X), joined["Hardness"])


## User-interface to test our predictions

In [ ]:
# TODO: slider to find the strongest correlation

## Export the results

## Try some Python commands yourself

In [ ]:
print('Hi, my name is ____.')

In [ ]:
# Creating variables:

temperature_in_F = 212.0
temperature_in_C = ...


## Next steps in learning Python